Primeramente vamos a inicializar los paquetes necesarios

In [ ]:
from autoscript_tem_microscope_client import TemMicroscopeClient
from autoscript_tem_microscope_client.enumerations import *
from autoscript_tem_microscope_client.structures import *

In [1]:
import os

In [8]:
directory = 'images/'
os.makedirs(directory, exist_ok=True)

In [ ]:
# Spiral Grid Generatod, copied from the Hackathon project
def build_spiral_coordinates(total_cells = 12):
    coord_initial = []
    directions = [(0, 1), (-1, 0), (0, -1), (1, 0)]
    direction_index = 0
    step_count = 0
    step_limit = 1
    direction_changes = 0

    while len(coord_initial) < total_cells:

        coord_initial.append(directions[direction_index])
        step_count += 1

        if step_count == step_limit:
            step_count = 0
            direction_index = (direction_index + 1) % 4
            direction_changes += 1
            
            if direction_changes % 2 == 0:
                step_limit += 1

    return coord_initial

In [ ]:
def movement(grid_x, grid_y, step_size, microscope):
    for i in range (0,2):
            microscope.specimen.stage.relative_move(StagePosition(x=grid_x * step_size, y=grid_y * step_size)) # Move the microscope to the coordinates of the grid
            image = microscope.acquisition.acquire_stem_image(DetectorType.HAADF, ImageSize.PRESET_512, 1e-6)
            image.save(os.path.join(directory,f"image_{grid_x}_{i}.png"))

In [ ]:
def main():
    microscope = TemMicroscopeClient()
    microscope.connect()
    microscope.optics.optical_mode = OpticalMode.STEM

    num_images = 1000
    step_size = 0.0001

    total_steps = num_images/2
    movement_direction = build_spiral_coordinates(total_cells=total_steps)

    for (x, y) in movement_direction:
        movement(x, y, step_size, microscope)
    
    microscope.disconnect()

In [ ]:
if __name__ == "__main__":
    main()

# Pruebas con prediccion

## Charge image

In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import label

In [1]:
def load_image(directory, index):
    filename = f"image_{index:04d}.png" # This will add ceros if the index is 5 until there are 4 numbers
    path = os.path.join(directory, filename)
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"No se ha encontrado la imagen: {path}")
    return image

In [3]:
# Función para aislar las partículas usando una máscara de predicción y conectar regiones.
def isolate_particles(pred_mask, threshold=0.5):
    # Convertir la predicción a una máscara binaria
    binary_mask = (pred_mask > threshold).astype(np.uint8)
    # Realizar el etiquetado de componentes conectados
    labeled_mask, num_particles = label(binary_mask)
    return labeled_mask, num_particles

In [4]:
# Función para calcular el cuadrado mínimo (cuadro envolvente) para una partícula
def get_square_bbox(indices, image_shape):
    rows, cols = indices
    if len(rows) == 0 or len(cols) == 0:
        return None
    min_row, max_row = np.min(rows), np.max(rows)
    min_col, max_col = np.min(cols), np.max(cols)
    width = max_col - min_col
    height = max_row - min_row
    side = max(width, height)
    
    # Calcular el centro para centrar el cuadrado
    row_center = (min_row + max_row) // 2
    col_center = (min_col + max_col) // 2
    half_side = side // 2

    start_row = max(row_center - half_side, 0)
    start_col = max(col_center - half_side, 0)
    
    # Asegurarse de que el cuadrado no exceda los límites de la imagen
    if start_row + side > image_shape[0]:
        start_row = image_shape[0] - side
    if start_col + side > image_shape[1]:
        start_col = image_shape[1] - side
    return (start_row, start_col, side)

In [5]:
# Función para plotear la partícula recortada de la imagen original
def plot_particle(image, bbox, title="Partícula"):
    r, c, s = bbox
    particle_img = image[r:r+s, c:c+s]
    plt.figure()
    plt.imshow(particle_img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# --- Código principal de ejemplo ---
# Define la carpeta donde están las imágenes y el índice a usar
folder_path = "ruta/a/la/carpeta"  # Cambia esto por la ruta real
index = 8  # Ejemplo del índice, para 'image_0008.png'

# Cargar la imagen
image = load_image(folder_path, index)

# Realizar la predicción con el modelo (asegúrate de tener el modelo cargado previamente)
pred, zones = model.predict(image)
plt.imshow(pred[0], cmap='gray')
plt.title("Predicción del modelo")
plt.axis('off')
plt.show()

# Aislar las partículas usando la máscara de predicción
labeled_mask, num_particles = isolate_particles(pred[0], threshold=0.5)
print(f"Número de partículas detectadas: {num_particles}")

# Convertir la imagen original a color para dibujar cuadros (en OpenCV el orden es BGR)
image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

bboxes = []
for i in range(1, num_particles + 1):
    # np.where devuelve las coordenadas (filas y columnas) donde la etiqueta es igual a i
    particle_indices = np.where(labeled_mask == i)
    bbox = get_square_bbox(particle_indices, image.shape)
    if bbox is not None:
        bboxes.append(bbox)
        # Dibujar el cuadrado en la imagen global; recordando que la coordenada para cv2.rectangle es (columna, fila)
        start_row, start_col, side = bbox
        cv2.rectangle(image_color, (start_col, start_row), (start_col + side, start_row + side), (0, 0, 255), 2)
        # Ploteamos cada partícula de forma individual
        plot_particle(image, bbox, title=f"Partícula {i}")

# Mostrar la imagen global con los cuadrados que encuadran las partículas
plt.figure(figsize=(8, 8))
# Convertimos de BGR a RGB para que matplotlib pinte con los colores correctos
plt.imshow(cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB))
plt.title("Imagen global con partículas enmarcadas")
plt.axis('off')
plt.show()